<a href="https://colab.research.google.com/github/eferna71/machinelearningcreditrisk/blob/main/machinelearningcreditrisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd, numpy as np

In [19]:
IN_DATAFILE="/lc_loan.csv"

In [20]:
loans = pd.read_csv(IN_DATAFILE, infer_datetime_format= True)


In [21]:
loans.dtypes

id                    int64
member_id             int64
loan_amnt           float64
funded_amnt         float64
funded_amnt_inv     float64
                     ...   
all_util            float64
total_rev_hi_lim    float64
inq_fi              float64
total_cu_tl         float64
inq_last_12m        float64
Length: 74, dtype: object

In [22]:
loans.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,NaN,NaN,3.0,0.0,13648.0,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-2015,171.62,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,NaN,3.0,0.0,1687.0,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,NaN,Sep-2013,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,NaN,NaN,2.0,0.0,2956.0,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-2014,649.91,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,35.0,NaN,10.0,0.0,5598.0,21.0,37.0,f,0.0,0.0,12226.302212,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-2015,357.48,NaN,Jan-2015,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,38.0,NaN,15.0,0.0,27783.0,53.9,38.0,f,766.9,766.9,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,Jan-2016,67.79,Feb-2016,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
datecols = ['issue_d','last_pymnt_d','next_pymnt_d','last_credit_pull_d','earliest_cr_line']

In [35]:
loans[datecols].dtypes

issue_d               object
last_pymnt_d          object
next_pymnt_d          object
last_credit_pull_d    object
earliest_cr_line      object
dtype: object

In [1]:
import datetime
def getDate(s):
  s=str(s)[0:8]
  retval=None
  try:
    retval=datetime.datetime.strptime(s, "%Y%m%d")
  except:
      pass
      #print s

  return retval

In [ ]:
%%time
newcols=['issue_date','final_date']
loans['issue_date']=loans['issue_date'].apply(getDate)
loans['final_date']=loans['final_d'].apply(getDate)

In [ ]:
newcols=['issue_date','final_date']
loans[datecols+newcols].head(10)

In [ ]:
%%time
loans['outcome']=(loans['final_date'] < datetime.datetime(2015,3,31))
loans['issue_year']=loans['issue_date'].apply(lambda x: x.year)
loans['rem_prnco'] = loans['funded_amnt'] - loans['total_rec_prncp']
loans['net_return'] = loans['total_pymnt'] - loans['funded_amnt']

Building credit scoring model

In [7]:
features = ['grade',
            'sub_grade_num',
            'short_emp',
            'emp_length_num',
            'homeownership',
            'dti',
            'purpose',
            'payment_inc_ratio',
            'deling_2yrs',
            'deling_2yrs_zero',
            'inq_last_6mnths',
            'last_delinq_none',
            'open_acc',
            'pub_rec',
            'pub_rec_zero',
            'revol_util',
            'total_rec_late_fee',
            ]
        
response= 'bad_loans'

In [ ]:
completed_loans=loans[(loans['outcome']==1) & (loans["inactive_loans"]==1)]


In [ ]:
clean_date=completed_loans[features+ [response]].dropna()

In [ ]:
completed_loans.shape,clean_data.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, confusion_matrix

import sklearn
from sklearn_pandas import DataFrameMapper

numerical_cols= ['sub_grade_num',
            'short_emp',
            'emp_length_num',
            'dti',
            'payment_inc_ratio',
            'deling_2yrs',
            'deling_2yrs_zero',
            'inq_last_6mnths',
            'last_delinq_none',
            'last_major_derog_none',
            'open_acc',
            'pub_rec',
            'pub_rec_zero',
            'revol_util',
            'total_rec_late_fee',]

categorical_cols=['grade', 'homeownership', 'purpose']

mapper = DataFrameMapper([
('grade',sklearn.preprocessing.LabelBinarizer()),
('homeownership',sklearn.preprocessing.LabelBinarizer()),
('purpose',sklearn.preprocessing.LabelBinarizer()),

    ])


X1=mapper.fit_transform(clean_data)
X1.shape

X2=np.array(clean_data[numerical_cols])
X2.shape



In [ ]:
X = np.hstack((X1,X2))
y = np.array(clean_data['bad_loans'])

X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state_100)

In [ ]:
rf_lm = LogisticRegression()
rf_lm.fit(X_train, y_train)

In [ ]:
y_pred = rf_lm.predict_proba(X_test)[:,1]
y_pred_class=rf_lm.predict(X_test)
accuracy = np.sum(y_pred_class==y_test)/y_test.shape[0]

print('accuracy = %0.2f' %accuracy)
